In [1]:
from IPython.lib.deepreload import original_import
%load_ext autoreload
%autoreload 2

from modular_ex4 import *
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
PADDING=2
GRAYSCALE = False
STEP_SIZE = 16
BORDER_CUT = 15

garden_video = "Garden.mp4"
house_video = "House.mp4"
kessaria_video = "Kessaria.mp4"
boat_video = "boat.mp4"

# 1. Load & Stabilize
raw = load_video_frames(house_video)
stable = stabilize_video(raw, step_size=STEP_SIZE, border_cut=BORDER_CUT, enable_rotation=True)

# 2. Compute Path
matrices = compute_camera_path(stable, step_size=STEP_SIZE, border_cut=BORDER_CUT)
geo = compute_canvas_geometry(matrices, raw.shape[1], raw.shape[2])

# 3. Render
panorama_left = render_strip_panorama(stable, matrices, geo, strip_anchor=0.2, strip_padding=PADDING, grayscale_out=GRAYSCALE)
panorama_mid = render_strip_panorama(stable, matrices, geo, strip_anchor=0.5, strip_padding=PADDING, grayscale_out=GRAYSCALE)
panorama_right = render_strip_panorama(stable, matrices, geo, strip_anchor=0.8, strip_padding=PADDING, grayscale_out=GRAYSCALE)
print(">>> done.")

In [ ]:
# show all 3 panoramas one line below the other
# with tight layout, no vertical padding between lines
plt.figure(figsize=(12, 12))
plt.subplot(3, 1, 1)
plt.imshow(panorama_left)
plt.title("Panorama Left Anchor 0.2")
plt.axis('off')
plt.subplot(3, 1, 2)
plt.imshow(panorama_mid)
plt.title("Panorama Mid Anchor 0.5")
plt.axis('off')
plt.subplot(3, 1, 3)
plt.imshow(panorama_right)
plt.title("Panorama Right Anchor 0.8")
plt.axis('off')
plt.tight_layout(pad=0.1)
plt.show()

In [ ]:
# 2. רשימה לשמירת הפריימים של הסרטון החדש
movie_frames = dynamic_mosaic(stable, matrices, geo)

In [ ]:
import imageio

imageio.mimsave('outputs/house_parallax_movie.mp4', movie_frames, fps=10)
print("Movie saved!")